In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# XGBoost, Regular Target Encoding - Loop the Blending Threshold
Back to standard target encoding, but different amounts of blending (lambda_k)

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import pickle

In [3]:
from pathlib import Path

In [4]:
import sklearn as sk
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance, partial_dependence
from scikitplot.metrics import plot_lift_curve

In [5]:
from sba_gnn.sba_gnn import sg_plot, sg_blender
from sba_gnn.sba_gnn.sg_blender import HierarchicalEncoder # Target encoder

## Import Processed Datasets

In [6]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet')) 

In [7]:
predictor_features = setup.predictor_features + ['menc_NAICS']
print(predictor_features)
with open(Path(setup.temp_path).joinpath('14_DATA_features_predict.pkl'), 'wb') as fout:
    pickle.dump(predictor_features, fout)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'menc_NAICS']


In [8]:
best_params_df = pd.read_csv(Path(setup.temp_path).joinpath('03_REPORT_fit_parameter_selected.csv'))
best_params = best_params_df.to_dict(orient='records')[0]
best_params

{'subsample': 0.8,
 'scale_pos_weight': 1.0,
 'reg_lambda': 0.01,
 'reg_alpha': 0.001,
 'min_child_weight': 50,
 'max_depth': 8,
 'learning_rate': 0.1,
 'gamma': 2}

In [9]:
xgb_params = dict(best_params, 
                   **{'objective':"binary:logistic", 'n_estimators':setup.xgb_n_estimators})

## Function to get metrics

In [10]:
def get_metrics(data, lambda_k = 100, enc_features = ['NAICS']):
    
    # Alternative encoding
    train_df = data[data['dset'] == 'train']
    hier_enc = HierarchicalEncoder(lambda_k = lambda_k)
    hier_enc.fit(train_df[enc_features],train_df['target'])
    hier_col = hier_enc.transform(data[enc_features]) \
        .rename('menc_NAICS')
    
    # Append to data
    data = pd.concat([data.drop(columns='menc_NAICS', errors='ignore'), 
                       hier_col], axis=1)
    train_df = data[data['dset'] == 'train']
    test_df = data[data['dset'] == 'test']
    val_df = data[data['dset'] == 'val']
    
    # Model fit
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                             n_estimators =  setup.xgb_n_estimators)
    xgb_model = xgb_model.set_params(**xgb_params)
    xgb_model.fit(train_df[predictor_features], train_df['target'],
              eval_set = [(train_df[predictor_features], train_df['target']),
                         (val_df[predictor_features], val_df['target'])])
    
    # Predict
    pred_df = pd.concat([data[['LoanNr_ChkDgt', 'dset', 'target', 'dset_naics_holdout']].reset_index(),
                     pd.DataFrame({'predict_prob':
                                   xgb_model.predict_proba(data[predictor_features])[:,1]})],
                    axis=1) \
    .set_index('index')
    
    # Decision threshold
    pred_train_df = pred_df[pred_df['dset'] == 'train']
    thresh_tune_data = sg_plot.get_f1_frame(pred_train_df['target'], 
                                        pred_train_df['predict_prob'])
    thresh_head = thresh_tune_data.sort_values('f1', ascending=False).head(2)
    best_thresh = thresh_head['thresh'].iloc[0]
    pred_df['predict_bin'] = np.where(pred_df['predict_prob'] >= best_thresh, 1, 0)
    
    # Metrics
    metrics_dset_df = pred_df.groupby('dset') \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_test_df = pred_df[pred_df['dset'] == 'test'] \
        .groupby(['dset', 'dset_naics_holdout']) \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
    
    return pred_df, metrics_df

## Loop to get predictions, metrics

In [11]:
count_sets = [5, 10, 20, 50, 100, 200, 500, 1000, 2000]

In [12]:
%%capture
pred_df = pd.DataFrame()
metrics_df = pd.DataFrame()
for c in count_sets:
    this_pred, this_met = get_metrics(sba_loans, c)
    this_pred['c'] = c
    pred_df = pd.concat([pred_df, this_pred])
    this_met['c'] = c
    metrics_df = pd.concat([metrics_df, this_met])

In [13]:
metrics_df[metrics_df['dset_naics_holdout'] == 1]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,c
1,test,0.660057,0.419184,0.303965,0.675070,0.346973,0.728879,1.0,5
1,test,0.658335,0.420655,0.303997,0.682599,0.347869,0.728881,1.0,10
1,test,0.648927,0.419973,0.300076,0.699433,0.348314,0.729411,1.0,20
1,test,0.640872,0.421819,0.298128,0.720920,0.346726,0.728902,1.0,50
1,test,0.637705,0.421559,0.296926,0.726504,0.346337,0.729154,1.0,100
1,test,0.626945,0.420064,0.292723,0.743507,0.346550,0.728087,1.0,200
1,test,0.678596,0.418609,0.311793,0.636748,0.347015,0.729066,1.0,500
1,test,0.644115,0.420457,0.298585,0.710431,0.349323,0.730202,1.0,1000
1,test,0.624362,0.420975,0.292387,0.751459,0.350303,0.730388,1.0,2000


In [14]:
metrics_df[metrics_df['dset_naics_holdout'] == 0]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,c
0,test,0.660692,0.449537,0.337337,0.673568,0.383545,0.728647,0.0,5
0,test,0.660179,0.449601,0.337110,0.674765,0.384233,0.728702,0.0,10
0,test,0.659237,0.449069,0.336409,0.675181,0.383440,0.728632,0.0,20
0,test,0.660082,0.449874,0.337184,0.675701,0.382783,0.728187,0.0,50
0,test,0.659365,0.449410,0.336624,0.675857,0.382950,0.728178,0.0,100
0,test,0.660457,0.448197,0.336625,0.670395,0.382382,0.727598,0.0,200
0,test,0.654957,0.447275,0.333538,0.678718,0.380409,0.726449,0.0,500
0,test,0.622835,0.446239,0.319656,0.738802,0.373725,0.722421,0.0,1000
0,test,0.617078,0.443747,0.316419,0.742548,0.365011,0.718185,0.0,2000


In [15]:
pred_df.to_parquet(Path(setup.temp_path).joinpath('14_DATA_combined_predictions.parquet'))
metrics_df.to_csv(Path(setup.temp_path).joinpath('14_REPORT_metrics.csv'), index=True)